# Capstone Project


### Note: This notebook will be used for the capstone project required for the IBM Data Science Specialization on coursera.

The goal of this notebook is, to cluster the neighborhoods of munich in order to find similar neighborhoods to live in.

In [1]:
# necessary imports
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Let's get the geographical coordinates of Munich.

In [4]:
address = 'Munich'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_munich = location.latitude
longitude_munich = location.longitude
print('The geograpical coordinate of Munich are {}, {}.'.format(latitude_munich, longitude_munich))

The geograpical coordinate of Munich are 48.1371079, 11.5753822.


Let's get the postal codes of munich in germany.

In [5]:
url = 'https://www.muenchen.de/int/en/living/postal-codes.html'
munich_data_list = pd.read_html(url)
munich_data = munich_data_list[0]
munich_data

,District,Postal Code
0,Allach-Untermenzing,"80995, 80997, 80999, 81247, 81249"
1,Altstadt-Lehel,"80331, 80333, 80335, 80336, 80469, 80538, 80539"
2,Au-Haidhausen,"81541, 81543, 81667, 81669, 81671, 81675, 81677"
3,Aubing-Lochhausen-Langwied,"81243, 81245, 81249"
4,Berg am Laim,"81671, 81673, 81735, 81825"
5,Bogenhausen,"81675, 81677, 81679, 81925, 81927, 81929"
6,Feldmoching-Hasenbergl,"80933, 80935, 80995"
7,Hadern,"80689, 81375, 81377"
8,Laim,"80686, 80687, 80689"
9,Ludwigsvorstadt-Isarvorstadt,"80335, 80336, 80337, 80469"


## Preprocessing

Let's start to preprocess the data in order to have a acceptable data format

First step: Split all places according to their postal codes

In [6]:
munich_data_cleaned = pd.DataFrame(columns=['District', 'Postal Code'])
munich_data_cleaned.head()

,District,Postal Code


In [7]:
items = []
for idx, codes in enumerate(munich_data['Postal Code']):
    code_list = codes.split(',')
    district = munich_data['District'][idx]
    for element in code_list:
        element = element.replace(' ', '')
        items.append({'District': district, 'Postal Code': element})

In [8]:
munich_data_cleaned = munich_data_cleaned.append(items)
munich_data_cleaned.head()

,District,Postal Code
0,Allach-Untermenzing,80995
1,Allach-Untermenzing,80997
2,Allach-Untermenzing,80999
3,Allach-Untermenzing,81247
4,Allach-Untermenzing,81249


Let's now fetch all latitude and longitude values for each Postal Code by using the Foursquare API

In [9]:
# credentials
CLIENT_ID = 'S3SWDZ4KH1KEXMKCPNHUQT0HDXWF2YHNZMYNE4UCNK0O2G1C' # your Foursquare ID
CLIENT_SECRET = 'PKGFEDIOKLA3RWYA5EY0SM2WDWU42MCTJPO1D012L2JVI1BO' # your Foursquare Secret
VERSION = '20200410' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3SWDZ4KH1KEXMKCPNHUQT0HDXWF2YHNZMYNE4UCNK0O2G1C
CLIENT_SECRET:PKGFEDIOKLA3RWYA5EY0SM2WDWU42MCTJPO1D012L2JVI1BO


In [10]:
# create new dataframe additionally containing the latitude and longitude values of each district and postal code mapping 
munich_data_ll = pd.DataFrame(columns=['District', 'Postal Code', 'Latitude', 'Longitude'])

# loop over all entries of old data frame and store according values
items = []
for idx, district in enumerate(munich_data_cleaned['District']):
    code = munich_data_cleaned['Postal Code'][idx]
    address = district + ', ' + code # to get format of address

    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    items.append({'District': district, 
                  'Postal Code': code,
                  'Latitude': latitude,
                  'Longitude': longitude})

In [11]:
munich_data_ll = munich_data_ll.append(items)
munich_data_ll.head()

,District,Postal Code,Latitude,Longitude
0,Allach-Untermenzing,80995,48.190034,11.468105
1,Allach-Untermenzing,80997,48.192790,11.484461
2,Allach-Untermenzing,80999,48.195994,11.457013
3,Allach-Untermenzing,81247,48.176884,11.476058
4,Allach-Untermenzing,81249,48.176884,11.476058


## Visualization

Let's now create a map of all districts in munich.

In [12]:
# create map of munich using latitude and longitude values
map_munich = folium.Map(location=[munich_data_ll["Latitude"].iloc[0], munich_data_ll["Longitude"].iloc[0]], zoom_start=11)

# add markers to map
for lat, lng, district in zip(munich_data_ll['Latitude'], munich_data_ll['Longitude'], munich_data_ll['District']):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_munich)  
    
map_munich

## District Exploration

Let's now explore all districts in munich by fetching venues in the near of each district with the help of the foursquare API.

In [15]:
# function for getting all venues of munich
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
munich_venues = getNearbyVenues(names=munich_data_ll['District'],
                                   latitudes=munich_data_ll['Latitude'],
                                   longitudes=munich_data_ll['Longitude']
                                  )

Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Allach-Untermenzing
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Altstadt-Lehel
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Au-Haidhausen
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Aubing-Lochhausen-Langwied
Berg am Laim
Berg am Laim
Berg am Laim
Berg am Laim
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Bogenhausen
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Feldmoching-Hasenbergl
Hadern
Hadern
Hadern
Laim
Laim
Laim
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Ludwigsvorstadt-Isarvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Maxvorstadt
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Milbertshofen-Am Hart
Moosach
Moosach
Moosach
Moosach
Moosach
Neuhausen-Nymphenburg
Neuhausen-Nym

In [17]:
# lets get the shape of the new dataframe
munich_venues.shape

(3561, 7)

In [18]:
# Lets visualize the head of the new dataframe
munich_venues.head()

,District,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allach-Untermenzing,48.190034,11.468105,Sport Bittl,48.191447,11.466553,Sporting Goods Shop
1,Allach-Untermenzing,48.190034,11.468105,Trattoria Olive,48.189905,11.466970,Trattoria/Osteria
2,Allach-Untermenzing,48.190034,11.468105,dm-drogerie markt,48.194118,11.465640,Drugstore
3,Allach-Untermenzing,48.190034,11.468105,Rossmann,48.193301,11.466388,Drugstore
4,Allach-Untermenzing,48.190034,11.468105,Hotel im Bunker,48.189208,11.466852,Hotel


Let's check how many venues were returned for each district

In [19]:
munich_venues.groupby('District').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Allach-Untermenzing,31,31,31,31,31,31
Altstadt-Lehel,623,623,623,623,623,623
Au-Haidhausen,259,259,259,259,259,259
Aubing-Lochhausen-Langwied,20,20,20,20,20,20
Berg am Laim,40,40,40,40,40,40
Bogenhausen,42,42,42,42,42,42
Feldmoching-Hasenbergl,15,15,15,15,15,15
Hadern,39,39,39,39,39,39
Laim,67,67,67,67,67,67


Let's find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 200 uniques categories.


## Analyze each District

Now lets analyze each district in order to get an idea of venues in the districts.

In [ ]:
# lets get a one hot encoding of all differen
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")

# add District column to dataframe
munich_onehot.insert(0, 'District', munich_data_ll['District'])
munich_onehot.head()

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
munich_grouped = munich_onehot.groupby('District').mean().reset_index()
munich_grouped.head(10)

Let's check the new size

In [ ]:
munich_grouped.shape

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in munich_grouped['District']:
    print("----"+hood+"----")
    temp = munich_grouped[munich_grouped['District'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] # exclude District column
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['District']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
district_venues_sorted = pd.DataFrame(columns=columns)
district_venues_sorted['District'] = munich_grouped['District']

for ind in np.arange(munich_grouped.shape[0]):
    district_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)

district_venues_sorted.head()

## Cluster Neighborhoods

Now that we have an overview about the data and made some first explorations, it's time to cluster the neighborhoods in order to get an idea about the types of neighborhoods and which district seems to be similar to which other districts.

In [ ]:
num_clusters = 5

X = munich_grouped.drop('District', 1)

kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(X)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
district_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

munich_merged = munich_data_ll

# merge labels and data about venues to district data and latitude plus longitude data to have all in one dataframe
munich_merged = munich_merged.join(district_venues_sorted.set_index('District'), on='District')

munich_merged.head()

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude_munich, longitude_munich], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i**4) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['Latitude'], munich_merged['Longitude'], munich_merged['District'], munich_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

As last step, each cluster shall be examined according to its most frequent venues. Plot all values within a table.

In [ ]:
munich_merged.loc[munich_merged['Cluster Labels'] == 0, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

In [ ]:
munich_merged.loc[munich_merged['Cluster Labels'] == 1, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

In [ ]:
munich_merged.loc[munich_merged['Cluster Labels'] == 2, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

In [ ]:
munich_merged.loc[munich_merged['Cluster Labels'] == 3, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

In [ ]:
munich_merged.loc[munich_merged['Cluster Labels'] == 4, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]